In [43]:
import dask.dataframe as dd
import pandas as pd
from huggingface_hub import hf_hub_download
import tqdm as notebook_tqdm
import matplotlib.pyplot as plt

# Исходные файлы загружены на ресурс https://huggingface.co/datasets
REPO_ID = "DmitriiMiptEdu/event_data"
adv = 'adv_data.csv'
event = 'event_data.csv'
users = 'users_data.csv'


In [24]:
# event_data = dd.read_csv(hf_hub_download(repo_id=REPO_ID, filename=event, repo_type="dataset"), low_memory=False,  blocksize=25e6, dtype={'event_status': 'object','user_phone': 'object'})
event_data = dd.read_csv(hf_hub_download(repo_id=REPO_ID, filename=event, repo_type="dataset"), low_memory=False,  dtype={'event_status': 'object','user_phone': 'object'})
adv_data = dd.read_csv(hf_hub_download(repo_id=REPO_ID, filename=adv, repo_type="dataset"), low_memory=False, )
users_data = dd.read_csv(hf_hub_download(repo_id=REPO_ID, filename=users, repo_type="dataset"),low_memory=False,  dtype={'registration_dtm': 'object', 'delete_dtm': 'object','user_phone': 'object'} )
print(f'\nevent_data.info():\n {event_data.info()}')
print(f'\n\nadv_data.info():\n {adv_data.info()}')
print(f'\n\nusers_data.info():\n {users_data.info()}')

<class 'dask_expr.DataFrame'>
Columns: 9 entries, id to product_type
dtypes: float64(1), string(8)
event_data.info():
 None
<class 'dask_expr.DataFrame'>
Columns: 9 entries, user_id to campaign_cost
dtypes: float64(1), string(8)

adv_data.info():
 None
<class 'dask_expr.DataFrame'>
Columns: 4 entries, user_phone to delete_dtm
dtypes: string(4)

users_data.info():
 None


In [35]:
event_data['user_phone'] = event_data['user_phone'].str.replace('+', '', regex=False)
event_data.head()

,id,user_phone,event_type,event_dtm,event_status,update_dtm,payout,currency,product_type
0,1263769bddda6eaaf7c8b8b22812ad6a063ad234d80782...,79140092454,show,2023-10-05 12:03:04.125806,NaN,NaT,NaN,RUB,mortgage
1,ae05865023d28ae72dd17955851263ead2d4daf11c69e0...,79125904946,show,2023-05-16 10:16:54.554523,NaN,NaT,NaN,RUB,loan
2,9861b00e9c9a99a6fdb2e5c11beebe9581eb1dda36ddc4...,79184556300,show,2023-12-23 22:46:44.066599,NaN,NaT,NaN,RUB,loan
3,349366af4823add107077299482f21031941fc6d4cd571...,79119287528,show,2023-07-30 05:25:46.285514,NaN,NaT,NaN,RUB,loan
4,2be2933460f58bb2d9a311ede6bb6ee064989568ab57b8...,79119287528,click,2023-07-30 05:25:46.285514,NaN,2023-07-30 05:42:59.014970,2.18,USD,loan


In [26]:
users_data.head()


,user_phone,user_id,registration_dtm,delete_dtm
0,79100004508,0000038524a303ed5876b823574848e0,2023-01-15 15:33:14.952938,NaN
1,79100005383,000003ef5bdfb2545c8d5ab693142e30,2022-11-21 22:43:54.418792,NaN
2,79100006444,0000056caffa28b60bf17bc69fe4366f,2022-12-23 03:00:04.088219,NaN
3,79100007221,00000a6f236f49c556e9a7b31d1b01e4,2022-12-23 03:31:50.935786,NaN
4,79100009619,00000cbd32277c99f2bd5e180a21f2b2,2023-01-25 14:31:17.234998,NaN


In [29]:
adv_data.head()

,user_id,campaign_type,campaign_start_dtm,campaign_end_dtm,source_medium,utm_campaign,interface,currency,campaign_cost
0,db00f9c699edacc50090172cfaa3b43a,show,2023-09-18 19:39:25.390618,2023-09-18 19:39:25.390618,finance.rambler.ru / cpm,rebrand_megamaket_reels,app,RUB,NaN
1,88aa22bca4fa1d6da73deb4b5c5ee8c0,show,2023-04-11 06:47:33.199369,2023-04-11 06:47:33.199369,yandex / organic,adv_retarget_brandkeys,web,RUB,NaN
2,e9e12347e50732804d2f8ded00743a07,show,2023-10-28 04:35:48.329870,2023-10-28 04:35:48.329870,yandex / organic,bk_light_brandkeys,web,RUB,NaN
3,fa0cd363d1978a27a078152cd263c670,show,2023-05-28 18:25:18.694051,2023-05-28 18:25:18.694051,yandex / organic,retarget_mind_response,api,RUB,NaN
4,fa0cd363d1978a27a078152cd263c670,click,2023-05-31 18:55:55.352680,2023-05-31 18:55:55.352680,regular / sms,light_clickarrow_clickarrow,api,USD,1.34


In [28]:
# Преобразование полей датасетов во временные метки
event_data['event_dtm'] = dd.to_datetime(event_data['event_dtm'])
event_data['update_dtm'] = dd.to_datetime(event_data['update_dtm'])
adv_data['campaign_start_dtm'] = dd.to_datetime(adv_data['campaign_start_dtm'])
adv_data['campaign_end_dtm'] = dd.to_datetime(adv_data['campaign_end_dtm'])
users_data['registration_dtm'] = dd.to_datetime(users_data['registration_dtm'])
users_data['delete_dtm'] = dd.to_datetime(users_data['delete_dtm'])


In [31]:
# Объединение users_data и event_data
merged_data = dd.merge(event_data, users_data, on='user_phone', how='left')

In [32]:
merged_data.head(10)

,id,user_phone,event_type,event_dtm,event_status,update_dtm,payout,currency,product_type,user_id,registration_dtm,delete_dtm
0,afcab77c4c5e2fa9b727b9f1e61b3cafcd45a9303af0b6...,+79130401253,show,2023-11-30 00:37:41.448284,NaN,NaT,NaN,USD,loan,4f0c1ae3a0b78de5c99b2f9e3b2f45d4,2023-03-03 12:30:52.961545,NaT
1,ff3c38d17c28cdb632a6c21987ed02fc89b743dfd6e3ef...,+79167048203,show,2023-02-21 12:36:51.685411,NaN,NaT,NaN,RUB,loan,ac45cd0f11505c804e521795f3ec323e,2022-11-23 19:13:19.899948,NaT
2,8e9b8c299ca7fd4d7d875b6c00176a5908229e845c9883...,+79172516240,show,2023-05-23 00:21:37.450295,NaN,NaT,NaN,RUB,loan,ba28cddf8a255634f4465ebdc4e85075,2023-02-19 08:40:37.400225,NaT
3,2625d13f78a78693b5e6c9d35f50472b93310dab3675be...,+79172516240,click,2023-05-23 00:21:37.450295,NaN,2023-05-23 00:41:44.756114,0.0,RUB,loan,ba28cddf8a255634f4465ebdc4e85075,2023-02-19 08:40:37.400225,NaT
4,990d9c792c514ac4281bad25021b67f04138b4ea7b49ce...,+79109871230,show,2024-01-02 11:31:34.419803,NaN,NaT,NaN,RUB,deposits,1ad891edaef7a9036fec20add80a57da,2023-05-02 08:35:54.354982,NaT
5,1f2bb0d4d9d6b70d9699d1d2af4541e53d10fa468bb7db...,+79187231775,show,2023-04-07 01:51:55.133452,NaN,NaT,NaN,RUB,loan,df8fcf2960dcee3edbda3016e6072d2d,2023-03-24 01:42:28.530330,NaT
6,a3b3155c2e7be705781ea5a487226e4666259393abf4b5...,+79188307017,show,2023-03-14 07:30:00.756048,NaN,NaT,NaN,USD,loan,e24a425afc4f80a3055244278028b80d,2022-10-27 23:43:35.675132,NaT
7,dbcfb4b67086f53b68d3e809e862f15bd93bcd2312c8e3...,+79115149884,show,2023-05-02 14:50:14.677964,NaN,NaT,NaN,RUB,loan,2852e320b78cb469bfd8d65c12170f92,2023-03-06 16:47:57.319057,NaT
8,26910cc028527983b1735a3171b2bd85c7e9ad851b4e48...,+79136911514,show,2023-04-15 02:45:54.218408,NaN,NaT,NaN,RUB,loan,5f99906750a713f68c4ac86d3d00fb5d,2023-02-10 10:48:08.961307,NaT
9,d54ae95f031c47deb5ddb33bcae744e36454d73ef52c84...,+79116878935,show,2023-07-22 05:20:07.486385,NaN,NaT,NaN,RUB,loan,2cbbabaf8f6ea3c79e0af73e4c7783c5,2023-03-07 21:13:57.623226,NaT


In [33]:

# Объединение с adv_data
merged_data = dd.merge(merged_data, adv_data, on='user_id', how='left')

In [34]:
merged_data.head(10)

,id,user_phone,event_type,event_dtm,event_status,update_dtm,payout,currency_x,product_type,user_id,registration_dtm,delete_dtm,campaign_type,campaign_start_dtm,campaign_end_dtm,source_medium,utm_campaign,interface,currency_y,campaign_cost
0,db2d3fb233f1a9dc8076bce0f7f7eb500a68f2e4328f05...,+79162021850,show,2024-01-03 21:10:11.133388,NaN,NaT,NaN,RUB,loan,9f839cfdf98473552f7d54be71817dea,2023-05-01 14:33:20.049524,NaT,show,2023-07-30 17:52:54.059633,2023-07-30 17:52:54.059633,yandex / organic,target_client_rebrand,app,RUB,NaN
1,1ed7b0f97e76477ac4920fe9f4f255e006c89228686690...,+79162021850,click,2024-01-03 21:10:11.133388,NaN,2024-01-03 21:17:48.178325,0.0,RUB,loan,9f839cfdf98473552f7d54be71817dea,2023-05-01 14:33:20.049524,NaT,show,2023-07-30 17:52:54.059633,2023-07-30 17:52:54.059633,yandex / organic,target_client_rebrand,app,RUB,NaN
2,c91d37836d8a5884eeb3b7a1ea77b864a76ad1fedd0591...,+79148227089,show,2023-02-28 03:34:31.492589,NaN,NaT,NaN,RUB,mortgage,7c6a806f145ba76cdcb1fec6b3b202f3,2022-11-18 12:36:05.213179,NaT,show,2023-06-21 10:21:50.755443,2023-06-21 10:21:50.755443,ios / mobile_app,light_market_adv,app,RUB,NaN
3,e9fdb7d5b88253427e159579c53e080c7f8a27f5f6b02e...,+79157796431,show,2023-04-16 06:26:18.296362,NaN,NaT,NaN,RUB,loan,94c3497d3909ee256f6aa01005c6a21f,2022-12-06 13:20:25.968799,NaT,show,2023-09-28 11:42:50.921042,2023-09-28 11:42:50.921042,google / organic,clickarrow_adv_clickarrow,app,RUB,NaN
4,e9fdb7d5b88253427e159579c53e080c7f8a27f5f6b02e...,+79157796431,show,2023-04-16 06:26:18.296362,NaN,NaT,NaN,RUB,loan,94c3497d3909ee256f6aa01005c6a21f,2022-12-06 13:20:25.968799,NaT,click,2023-11-26 00:40:58.413915,2023-11-26 00:40:58.413915,yandex / organic,mind_bk_client,web,RUB,0.0
5,6a4de0e4553bab1a743e0df48f572105f7b520a5055a2f...,+79168865804,show,2023-03-01 06:22:43.940051,NaN,NaT,NaN,RUB,loan,b0dec61e1a92f2d728b4fb09096ed132,2022-08-29 00:55:03.467320,NaT,show,2022-12-13 19:43:10.802567,2022-12-13 19:43:10.802567,yandex / organic,market_retarget_case,app,RUB,NaN
6,6a4de0e4553bab1a743e0df48f572105f7b520a5055a2f...,+79168865804,show,2023-03-01 06:22:43.940051,NaN,NaT,NaN,RUB,loan,b0dec61e1a92f2d728b4fb09096ed132,2022-08-29 00:55:03.467320,NaT,click,2023-01-25 06:42:41.672197,2023-01-25 06:42:41.672197,yandex / cpc,light_case_reels,app,RUB,0.0
7,a7c42af8a80ce5b364bcd7963492caeacbe8faf2d9101a...,+79128114885,show,2023-07-13 07:47:54.033235,NaN,NaT,NaN,RUB,mortgage,493b8d37c80db36eec1a5b24e0d29170,2023-07-01 23:08:24.270742,NaT,show,2023-09-01 12:27:02.810640,2023-09-01 12:27:02.810640,google / organic,rebrand_market_bk,web,RUB,NaN
8,634460106c688ed12d9ce5508a9582cb411c9ef342c9cb...,+79100674773,show,2023-06-09 02:18:41.735490,NaN,NaT,NaN,RUB,loan,0371763d0902c11970bf60d8b422c873,2023-03-25 02:21:30.148443,NaT,show,2023-05-08 14:30:00.306183,2023-05-08 14:30:00.306183,yandex / cpc,light_client_mind,app,USD,NaN
9,634460106c688ed12d9ce5508a9582cb411c9ef342c9cb...,+79100674773,show,2023-06-09 02:18:41.735490,NaN,NaT,NaN,RUB,loan,0371763d0902c11970bf60d8b422c873,2023-03-25 02:21:30.148443,NaT,click,2023-05-11 18:37:58.644710,2023-05-11 18:37:58.644710,yandex / cpc,operator_client_title,api,RUB,0.0


In [36]:
# Анализ событий пользователей
# Сортировка событий по пользователям и времени
merged_data = merged_data.sort_values(by=['user_phone', 'event_dtm'])

In [37]:
# Идентификация конверсий
conversions = merged_data[merged_data['event_status'] == 'success']

# Разработка модели атрибуции

In [38]:
# Реализация модели линейной атрибуции
def linear_attribution(events):
    n = len(events)
    attribution = 1.0 / n
    events['attribution'] = attribution
    return events

# Применение линейной модели атрибуции
attributed_data = merged_data.groupby('user_phone').apply(linear_attribution, meta=merged_data)



In [44]:
attributed_data.head()

# Визуализация результатов

In [ ]:
# Сбор данных для визуализации
source_attribution = attributed_data.groupby('source_medium')['attribution'].sum().compute()
print(source_attribution)

# Визуализация результатов
source_attribution_pd = source_attribution.compute()

# Пример простой визуализации распределения атрибуции
source_attribution_pd.plot(kind='bar')
plt.title('Атрибуция по источникам трафика')
plt.xlabel('Источник')
plt.ylabel('Атрибуция')
plt.show()